In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

## TASK 1: 
### Request the Falcon9 Launch Wiki page from its URL.

In [3]:
url = 'https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922'
response = requests.get(url)
response.status_code

200

In [15]:
soup = BeautifulSoup(response.content, 'html5lib')

In [16]:
print('Title is:',soup.title.string)

Title is: List of Falcon 9 and Falcon Heavy launches - Wikipedia


## TASK 2:
### Extract all column/variable names from the HTML table header.

In [31]:
table = soup.find_all('table')
print(f"Number of tables found: {len(table)}")

Number of tables found: 26


In [41]:
table = table[2]
print(table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

In [43]:
#headers = table.find_all('th')
column_names = []
headers = table.find_all('th')
for header in headers:
    column_names.append(header.get_text(separator=" ", strip=True))
    
print(column_names)

['Flight No.', 'Date and time ( UTC )', 'Version, Booster [ b ]', 'Launch site', 'Payload [ c ]', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing', '1', '2', '3', '4', '5', '6', '7']


## TASK 3: 
### Create a data frame by parsing the launch HTML tables.

In [76]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( UTC )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [78]:
# Assuming column_names is defined somewhere before this snippet
column_names = [
    'Flight No.', 'Date and time ( UTC )', 'Launch site', 'Payload',
    'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 
    'Version Booster', 'Booster landing'
]

# Initialize the launch_dict
launch_dict = dict.fromkeys(column_names)

# Remove the irrelevant column
if 'Date and time ( UTC )' in launch_dict:
    del launch_dict['Date and time ( UTC )']

# Initialize launch_dict with each value as an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added new columns
launch_dict['Version Booster'] = []
launch_dict['Booster landing'] = []
launch_dict['Date'] = []
launch_dict['Time'] = []

# Optional: Print the initial structure of launch_dict to verify
print(launch_dict)

# Next, you would typically follow this with your extraction code
# The extraction code should populate the above lists based on your web scraping logic

# Example of the extraction code from previous messages (assumed to be included)
# Extract each table 
extracted_row = 0
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            flight_number = rows.th.string.strip() if rows.th.string else None
            flag = flight_number.isdigit() if flight_number else False
        else:
            flag = False
        
        # Get table elements 
        row = rows.find_all('td')
        
        # If it is a number, save cells in a dictionary 
        if flag and row:  # Ensure that row has data
            extracted_row += 1
            
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)
            
            # Extract date and time
            datatimelist = date_time(row[0].get_text(separator=" ", strip=True))  # Clean the string
            date = datatimelist[0].strip(',') if datatimelist[0] else "N/A"
            launch_dict['Date'].append(date)
            
            # Time value
            time = datatimelist[1] if len(datatimelist) > 1 else "N/A"
            launch_dict['Time'].append(time)
              
            # Booster version
            bv = booster_version(row[1]) if row[1] else None
            if not bv and row[1].a:  # Check if anchor tag exists
                bv = row[1].a.string
            launch_dict['Version Booster'].append(bv if bv else "N/A")
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else "N/A"
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            payload = row[3].a.string if row[3].a else "N/A"
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4]) if row[4] else "N/A"
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            orbit = row[5].a.string if row[5].a else "N/A"
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            customer = row[6].a.string if row[6].a else "N/A"
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else "N/A"
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8]) if len(row) > 8 else "N/A"
            launch_dict['Booster landing'].append(booster_landing)

# Optional: Print the dictionary to check if data was populated correctly
print(launch_dict)


{'Flight No.': [], 'Launch site': [], 'Payload': [], 'Payload mass': [], 'Orbit': [], 'Customer': [], 'Launch outcome': [], 'Version Booster': [], 'Booster landing': [], 'Date': [], 'Time': []}
Error parsing date and time: time data '4 June 2010, 18:45' does not match format '%B %d, %Y %I:%M %p'


NameError: name 'booster_version' is not defined

In [80]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [82]:
df.to_csv('spacex_web_scraped.csv', index=False)